In [ ]:
import ccxt
import time
import pandas as pd
from tqdm.notebook import tqdm
# Defining assets
CRYPTO_LIST = ['BTC/USDT', 'ETH/USDT', 'BNB/USDT', 'SOL/USDT', 'BUSD/USDT']

FREQ = '4h'
LIMIT = 1000
FOLDER = ''
API_KEY = ''
SECRET_KEY = ''

ohlcvs = []

exchange = ccxt.binance({"apiKey": API_KEY, "secret": SECRET_KEY, 'enableRateLimit': True})

START_TIMES = [exchange.parse8601('2020-10-17T08:00:00Z'),exchange.parse8601('2021-04-02T00:00:00Z'),
               exchange.parse8601('2021-09-15T16:00:00Z'),exchange.parse8601('2022-03-01T08:00:00Z'),
               exchange.parse8601('2022-08-15T00:00:00Z'),exchange.parse8601('2023-01-28T16:00:00Z'),
               exchange.parse8601('2023-07-14T08:00:00Z')]

def get_ready_data(freq, crypto_list, limit, start_time):

    dfs = []

    for i in range(len(crypto_list)):

        TICKER = crypto_list[i]

        try:
            bars = exchange.fetch_ohlcv(TICKER, timeframe=freq, limit=limit, since=start_time)
            df = pd.DataFrame(bars, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
            df['ticker'] = TICKER
            dfs.append(df)
            time.sleep(1)

        except Exception as e:
            print(f"ticker {TICKER} : {str(e)}")

    ohlcv_df = pd.concat(dfs)
    ohlcv_df = ohlcv_df[['timestamp', 'ticker', 'close', 'open', 'high', 'low', 'volume']]
    ohlcv_df.columns = ['date', 'ticker', 'close', 'open', 'high', 'low', 'volume']
    ohlcv_df['date'] = pd.to_datetime(ohlcv_df['date'], unit='ms')
    ohlcv_df = ohlcv_df.set_index(["date", "ticker"]).sort_index().reset_index()

    return ohlcv_df

for t in tqdm(START_TIMES):
    data_kwargs = {
        'freq': FREQ,
        'crypto_list': CRYPTO_LIST,
        'limit': LIMIT,
        'start_time': t
    }

    ohlcv = get_ready_data(**data_kwargs)
    ohlcv['date'] = ohlcv['date'].apply(lambda x:str(x))
    ohlcvs.append(ohlcv)

ohlcv = pd.concat(ohlcvs).reset_index(drop=True).set_index(['date','ticker']).sort_index().reset_index()
ohlcv.to_csv(f'{FOLDER}ohlcv.csv', index=False)
ohlcv

  0%|          | 0/7 [00:00<?, ?it/s]

,date,ticker,close,open,high,low,volume
0,2020-10-17 08:00:00,BNB/USDT,30.4400,30.2351,30.5640,30.1563,2.966816e+05
1,2020-10-17 08:00:00,BTC/USDT,11343.1300,11344.9900,11358.8800,11308.6300,3.658009e+03
2,2020-10-17 08:00:00,BUSD/USDT,0.9996,0.9994,0.9996,0.9994,6.915902e+06
3,2020-10-17 08:00:00,ETH/USDT,368.1200,367.7900,368.4700,366.2900,3.795158e+04
4,2020-10-17 08:00:00,SOL/USDT,2.2145,2.2241,2.2462,2.2025,7.697393e+04
...,...,...,...,...,...,...,...
33085,2023-10-25 04:00:00,BNB/USDT,224.8000,227.2000,228.1000,224.0000,8.444686e+04
33086,2023-10-25 04:00:00,BTC/USDT,33832.6100,33999.3900,34204.7800,33679.0500,6.704907e+03
33087,2023-10-25 04:00:00,BUSD/USDT,0.9998,1.0000,1.0000,0.9998,1.066141e+07
33088,2023-10-25 04:00:00,ETH/USDT,1767.2000,1788.6700,1795.0000,1761.1700,5.197910e+04


In [ ]:
ohlcv['ticker'].value_counts()

BNB/USDT     6618
BTC/USDT     6618
BUSD/USDT    6618
ETH/USDT     6618
SOL/USDT     6618
Name: ticker, dtype: int64